In [4]:
!conda install -c conda-forge Skater --yes

Solving environment: done

# All requested packages already installed.



In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
# Reference for customizing matplotlib: https://matplotlib.org/users/style_sheets.html
plt.style.use('ggplot')

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

from skater.core.explanations import Interpretation
from skater.model import InMemoryModel


data = load_breast_cancer()
# Description of the data
print(data.DESCR)

pd.DataFrame(data.target_names)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

,0
0,malignant
1,benign


In [85]:
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')
eclf = eclf.fit(X_train, y_train)

clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)
clf3 = clf3.fit(X_train, y_train)

models = {'lr':clf1, 
          'rf':clf2, 
          'gnb':clf3, 
          'ensemble':eclf}

In [86]:
interpreter = Interpretation(X_test, feature_names=data.feature_names)

2018-11-20 15:50:14,178 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progress_bar=False


[30/30] features ████████████████████ Time elapsed: 0 seconds

2018-11-20 15:50:14,805 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progress_bar=False


[30/30] features ████████████████████ Time elapsed: 0 seconds

2018-11-20 15:50:15,453 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progress_bar=False


[30/30] features ████████████████████ Time elapsed: 0 seconds

2018-11-20 15:50:16,035 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progress_bar=False


[30/30] features ████████████████████ Time elapsed: 0 seconds

In [87]:
# Ensemble Classifier does not have feature importance enabled by default
f, axes = plt.subplots(2, 2, figsize = (26, 18))

ax_dict = {
    'lr':axes[0][0],
    'rf':axes[1][0],
    'gnb':axes[0][1],
    'ensemble':axes[1][1]
}

interpreter = Interpretation(X_test, feature_names=data.feature_names)

for model_key in models:
    pyint_model = InMemoryModel(models[model_key].predict_proba, examples=X_test)
    ax = ax_dict[model_key]
    interpreter.feature_importance.plot_feature_importance(pyint_model, ascending=True, ax=ax)
    ax.set_title(model_key)

In [88]:
# Before interpreting, lets check on the accuracy of all the models
from sklearn.metrics import f1_score
for model_key in models:
        print("Model Type: {0} -> F1 Score: {1}".
              format(model_key, f1_score(y_test, models[model_key].predict(X_test))))

Model Type: lr -> F1 Score: 0.974025974025974
Model Type: rf -> F1 Score: 0.974025974025974
Model Type: gnb -> F1 Score: 0.9426751592356688
Model Type: ensemble -> F1 Score: 0.967741935483871


In [89]:
def understanding_interaction():
    pyint_model = InMemoryModel(eclf.predict_proba, examples=X_test, target_names=data.target_names)
    # ['worst area', 'mean perimeter'] --> list(feature_selection.value)
    interpreter.partial_dependence.plot_partial_dependence(list(feature_selection.value),
                                                                pyint_model, 
                                                                grid_resolution=grid_resolution.value, 
                                                                with_variance=True)
        
    # Lets understand interaction using 2-way interaction using the same covariates
    # feature_selection.value --> ('worst area', 'mean perimeter')
    axes_list = interpreter.partial_dependence.plot_partial_dependence([feature_selection.value],
                                                                       pyint_model, 
                                                                       grid_resolution=grid_resolution.value, 
                                                                       with_variance=True)

In [90]:
#!conda install ipywidgets --yes
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

# One could further improve this by setting up an event callback using
# asynchronous widgets
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display
from IPython.display import clear_output
grid_resolution = widgets.IntSlider(description="GR", 
                                    value=10, min=10, max=100)
display(grid_resolution)

# dropdown to select relevant features from the dataset
feature_selection = widgets.SelectMultiple(
    options=tuple(data.feature_names),
    value=['worst area', 'mean perimeter'],
    description='Features',
    layout=widgets.Layout(display="flex", flex_flow='column', align_items = 'stretch'),
    disabled=False,
    multiple=True
)
display(feature_selection)

IntSlider(value=10, description='GR', min=10)

SelectMultiple(description='Features', index=(23, 2), layout=Layout(align_items='stretch', display='flex', fle…

2018-11-20 15:50:25,209 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progressbar=False


[10/10] grid cells ████████████████████ Time elapsed: 0 seconds

2018-11-20 15:50:25,739 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progressbar=False


[10/10] grid cells ████████████████████ Time elapsed: 0 seconds

2018-11-20 15:50:26,159 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progressbar=False


[100/100] grid cells ████████████████████ Time elapsed: 4 seconds

In [91]:
# Reference: http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

button = widgets.Button(description="Generate Interactions")
display(button)

def on_button_clicked(button_func_ref):
    clear_output()
    understanding_interaction()

button.on_click(on_button_clicked)

In [126]:
from skater.core.local_interpretation.lime.lime_tabular import LimeTabularExplainer
from IPython.display import display, HTML, clear_output

def on_value_change(change):
    index = 33
    exp = LimeTabularExplainer(X_test, 
                           feature_names=data.feature_names, 
                           class_names=['p(Cancer)', 'p(No Cancer)'])
    print("Model behavior at row: {}".format(index))
    # Lets evaluate the prediction from the model and actual target label
    print("prediction from the model:{}".format(eclf.predict(X_test[index].reshape(1, -1))))
    print("Target Label on the row: {}".format(y_test.reshape(1,-1)[0][index]))
    clear_output()
    display(HTML(exp.explain_instance(X_test[index], models['ensemble'].predict_proba).as_html()))

In [127]:
int_range = widgets.IntSlider(description="Index Selector", value=9, min=0, max=100)
display(int_range)
int_range.observe(on_value_change, names='value')

IntSlider(value=9, description='Index Selector')